In [1]:
import numpy as np # linear algebrae
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression

import os

# from polire import IDW

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from datetime import date, timedelta
import datetime

import json
import pickle as pkl

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import MSELoss

from PIL import Image
from torchvision.transforms import ToTensor, Resize
import torchvision.transforms as transforms

from tqdm import tqdm

In [2]:
class CFG:
    evaluation_time_gap = 1
    convert_numpy = False
    target_list = ['pm2_5', 'pm10']
    img_cols = ['lat_i', 'lon_j']
    features = ['timeOfDay', 'lat', 'lon', 'distance', 'bus_count', 'day_of_week', 'pm2_5', 'pm10']

    feature_indices = [0, 1, 2, 3, 4, 5, 7, 8]
    mask_index = 6

    batch_size = 4
    num_epochs = 50

    hidden_dim = 64
    num_layers = 4
    kernel_size = 3
    dropout=0.2
    
    lr = 2e-3
    patience=2
    factor=0.7
    warmup_steps = 15
    total_steps = num_epochs
    base_lr = lr
    final_lr = 5e-6

    grid_size = 30
    day_len = 35
    forecast_horizon = 35

    model_kind="gru"
    bidirectional=False
        
    pm2_5_thresholds = [0, 30, 60, 90, 120, 250, 2000]
    pm10_thresholds = [0, 50, 100, 250, 350, 430, 2000]
    aqi_category = ["Good", "Satisfactory", "Moderate", "Poor", "Very Poor", "Severe"]
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset

In [3]:
ds = np.load('/kaggle/input/airdelhi-baselines-deepengineering/idw_dense_grid.npz', allow_pickle=True)

In [4]:
# ds.files
df = ds['grids']
k = ds['keys']

In [5]:
rolled_df = np.roll(df, shift=35*7, axis=0)
df = np.concatenate([df, rolled_df[:, :2]], axis = 1)

In [6]:
df[0, :, 0, 0], df[35*7, :, 0, 0]

(array([2.2127020e+02, 2.4173746e+02, 0.0000000e+00, 6.0000000e+00,
        4.4699025e-04, 0.0000000e+00, 1.0000000e+00, 2.4057486e+02,
        2.6138818e+02], dtype=float32),
 array([2.3520129e+02, 2.5652615e+02, 0.0000000e+00, 6.0000000e+00,
        2.1018749e-03, 0.0000000e+00, 1.0000000e+00, 2.2127020e+02,
        2.4173746e+02], dtype=float32))

In [7]:
df[:7*35, 7:, :, :] = np.mean(df[:7*35, :2], axis = 0)

In [8]:
np_df = df

In [9]:
np_df.shape

(3185, 9, 30, 30)

In [10]:
feature_indices = [2, 3, 4, 5]
def min_max_scale(d):
    d = (d - d.min()) / (d.max() - d.min())
    return d
for f in feature_indices:
    np_df[:, f, :, :] = min_max_scale(np_df[:, f, :, :])    

In [11]:
# satellite_image = Image.open('/kaggle/input/delhi-satellite-images-grid/full_region_cropped.png')
# satellite_image
# np.array(satellite_image).shape
# satellite_image = ToTensor()(satellite_image)
# satellite_image = Resize(size=(300, 300))(satellite_image)
# satellite_image.shape

In [12]:
class ConvImageSequenceDataset(Dataset):

    def __init__(self, df, img_dict, feature_indices = None, img_cols = None, sequence_length=70, forecast_horizon=35):
        
        feature_indices = CFG.feature_indices if feature_indices is None else feature_indices
        img_cols = CFG.img_cols if img_cols is None else img_cols
        
        self.feature_indices = feature_indices
        self.img_dict = img_dict
        self.img_cols = img_cols
        
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon

        self.split_df(df)

        self.proc_img(img_dict)

    def proc_img(self, img_dict):
        self.imgs = []

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

        for i in range(CFG.grid_size):
            # self.imgs.append([])
            for j in range(CFG.grid_size):
                img_file = img_dict[(i, j)]
                img = Image.open(img_file).convert("RGB")
                tensor_img = transform(img)
                # self.imgs[-1].append(tensor_img)
                self.imgs.append(tensor_img)

        self.imgs = torch.stack(self.imgs).to(device) # (900, 3, 32, 32)

    def split_df(self, df):
        self.X = torch.tensor(df[:, self.feature_indices, :, :]).to(device)
        # originally, mask = 1 if it is NOT missing
        self.masks = 1 - torch.tensor(df[:, CFG.mask_index, :, :]).to(device).unsqueeze(1) 

        self.total_len = len(self.X) // CFG.day_len - self.sequence_length // CFG.day_len
        self.total_len = self.total_len - self.forecast_horizon // CFG.day_len + 1
    
    def __len__(self):
        return self.total_len

    def get_index(self, i):
        start = CFG.day_len * i
        end   = self.sequence_length + start
        test_end = self.forecast_horizon + end
        return start, end, test_end
    
    def __getitem__(self, i):
        start, end, test_end = self.get_index(i)
        X = self.X[start:end]# .squeeze(1)
        y = self.X[end:test_end, :2]# .squeeze(1)
        masks = self.masks[end:test_end]
        return X, y, masks


In [13]:
df = pd.read_csv('/kaggle/input/airdelhi-baselines-deepengineering/dense_df.csv')

df = df.drop(columns = 'Unnamed: 0')
df['date_value'] = pd.to_datetime(df['date_value'])

dates = pd.to_datetime([df['date_value'].min(), df['date_value'].max()])

max_train_date = dates.min() + (dates.max() - dates.min()) * 0.75
max_train_date = max_train_date.floor("D")
min_train_date = df['date_value'].min()
max_date = df['date_value'].max().floor("D")

metrics_dict = {
    'MSE': mean_squared_error, 
    'r2 score': r2_score, 
    'MAE': mean_absolute_error,
}

target_list = CFG.target_list

features = ['date_value', 'timeOfDay', 'lat', 'lon', 'day_of_week', 'distance', 'bus_count']

CFG.base_features = ['timeOfDay', 'lat', 'lon', 'day_of_week', 'distance', 'bus_count']
CFG.features = CFG.base_features

min_train_date, max_train_date

(Timestamp('2020-11-01 00:00:00'), Timestamp('2021-01-07 00:00:00'))

In [14]:
lat = df['lat'].unique()
lon = df['lon'].unique()

In [15]:
lat = np.sort(lat)
lon = np.sort(lon)

min_dist_lat = 1000
for i in range(len(lat) - 1):
    min_dist_lat = min(min_dist_lat, lat[i+1] - lat[i])
print(min_dist_lat)

for i in range(len(lat)):
    assert (lat[i] / min_dist_lat - round(lat[i] / min_dist_lat)) < 1e-6

min_dist_lon = 1000
for i in range(len(lon) - 1):
    min_dist_lon = min(min_dist_lon, lon[i+1] - lon[i])
print(min_dist_lon)

for i in range(len(lon)):
    assert (lon[i] / min_dist_lon - round(lon[i] / min_dist_lon)) < 1e-6

0.0384615384615188
0.04761904761903679


In [16]:
grid_dimensions = round(lat[-1] / min_dist_lat), round(lon[-1] / min_dist_lon)
grid_dimensions

(21, 21)

In [17]:
def add_lag_features(df, lags = [1]):

    df = df.copy()
    
    df.sort_values(by=["lat", "lon", "timeOfDay", "date_value"], inplace=True)

    added_features = []
    for l in lags:
        df[f'pm2_5_lag_{l}'] = df.groupby(
            ['timeOfDay', 'lat', 'lon'])['pm2_5'].shift(l)
        df[f'pm10_lag_{l}'] = df.groupby(
            ['timeOfDay', 'lat', 'lon'])['pm10'].shift(l)

        added_features.append(f'pm2_5_lag_{l}')
        added_features.append(f'pm10_lag_{l}')

        df.sort_values(by=["lat", "lon", "date_value"], inplace=True)

    # Group by latitude and longitude
    grouped = df.groupby(["lat", "lon"])

    # Function to fill NaN values based on previous mean
    def fill_na_with_previous_mean(group):
        for col in group.columns:
            if col not in ["date_value", "lat", "lon"]:
                group[col] = group[col].astype(float)  # Ensure numeric columns
                group[col] = group[col].fillna(group[col].expanding().mean().shift())  # Previous days' mean
                
                # If still NaN (first row), replace with overall mean
                overall_mean = df[col].mean(skipna=True)
                group[col] = group[col].fillna(overall_mean)
        return group

    # Apply the function to each group
    df = grouped.apply(fill_na_with_previous_mean)
    df.reset_index(drop=True, inplace=True)
    df = df.sort_values(by = ['date_value', 'timeOfDay', 'lat', 'lon'])
    df.reset_index(drop=True, inplace=True)
    CFG.features += added_features
    return df
    
df = add_lag_features(df, lags = [7])
df = df.sort_values(by=['lat', 'lon', 'timeOfDay', 'date_value'])

<ipython-input-17-5828f686307e>:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = grouped.apply(fill_na_with_previous_mean)


In [18]:
class CustomScaler:

    def __init__(self, df):
        self.pm2_5_max = float(df['pm2_5'].max())
        self.pm2_5_min = float(df['pm2_5'].min())
        
        self.pm10_max = float(df['pm10'].max())
        self.pm10_min = float(df['pm10'].min())

    def transform_numpy(self, X, target = 'pm2_5'):
        if target == 'pm2_5':
            X = (X - self.pm2_5_min) / (self.pm2_5_max - self.pm2_5_min)
        else:
            X = (X - self.pm10_min) / (self.pm10_max - self.pm10_min)

        return X

    def transform(self, df):
        df = df.copy()
        df['pm2_5'] = (df['pm2_5'] - self.pm2_5_min) / (self.pm2_5_max - self.pm2_5_min)
        df['pm10'] = (df['pm10'] - self.pm10_min) / (self.pm10_max - self.pm10_min)
        return df

    def inverse_transform(self, X):
        X[:, 0] = X[:, 0] * (self.pm2_5_max - self.pm2_5_min) + self.pm2_5_min
        X[:, 1] = X[:, 1] * (self.pm10_max - self.pm10_min) + self.pm10_min
        return X

target_scaler = CustomScaler(df)
df = target_scaler.transform(df)

CFG.features = ['timeOfDay', 'lat', 'lon', 'distance', 'bus_count', 
                'day_of_week', 'pm2_5', 'pm10', 'pm2_5_lag_7', 'pm10_lag_7']

df['day_of_week'] = df['date_value'].dt.dayofweek

scaler = MinMaxScaler()
df[CFG.features] = scaler.fit_transform(df[CFG.features])

In [19]:
np_df[:, 0, :, :] = target_scaler.transform_numpy(np_df[:, 0, :, :], 'pm2_5')
np_df[:, 1, :, :] = target_scaler.transform_numpy(np_df[:, 1, :, :], 'pm10')
np_df[:, 7, :, :] = target_scaler.transform_numpy(np_df[:, 7, :, :], 'pm2_5')
np_df[:, 8, :, :] = target_scaler.transform_numpy(np_df[:, 8, :, :], 'pm10')

In [20]:
with open("/kaggle/input/delhi-satellite-images-grid/tile_metadata.json", 'r') as f:
    metadata = json.load(f)

def process_metadata(metadata):
    img_dict = {}
    for k, v in metadata.items():
        i = v['row']
        j = v['col']
        img_dict[(i, j)] = f"/kaggle/input/delhi-satellite-images-grid/tiles_32x32/tile_{i}_{j}.png"

    return img_dict

img_dict = process_metadata(metadata)

lat = df['lat'].unique()
lon = df['lon'].unique()

lat = np.sort(lat)
lon = np.sort(lon)

min_dist_lat = 1000
for i in range(len(lat) - 1):
    min_dist_lat = min(min_dist_lat, lat[i+1] - lat[i])
print(min_dist_lat)

for i in range(len(lat)):
    assert (lat[i] / min_dist_lat - round(lat[i] / min_dist_lat)) < 1e-6

min_dist_lon = 1000
for i in range(len(lon) - 1):
    min_dist_lon = min(min_dist_lon, lon[i+1] - lon[i])
print(min_dist_lon)

for i in range(len(lon)):
    assert (lon[i] / min_dist_lon - round(lon[i] / min_dist_lon)) < 1e-6

def rounded_lat_lon(df):
    df = df.copy()
    df['lat_i'] = np.round(df['lat'] / min_dist_lat) + 2
    df['lon_j'] = np.round(df['lon'] / min_dist_lon) + 4
    return df

df = rounded_lat_lon(df)

0.04999999999997419
0.05263157894735837


In [21]:
max_train_index = 0
for i in range(len(k)):
    if k[i][0] == max_train_date:
        max_train_index = i
        break

In [22]:
train_ds = ConvImageSequenceDataset(np_df[:max_train_index, :, :, :], img_dict)
test_ds = ConvImageSequenceDataset(np_df[max_train_index:, :, :, :], img_dict)

train_dl = DataLoader(train_ds, batch_size = CFG.batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size = CFG.batch_size, shuffle=True)

In [23]:
xt, yt, mt = train_ds[0]
xt.shape, yt.shape, mt.shape

(torch.Size([70, 8, 30, 30]),
 torch.Size([35, 2, 30, 30]),
 torch.Size([35, 1, 30, 30]))

In [24]:
train_ds.imgs.shape

torch.Size([900, 3, 32, 32])

In [25]:
train_ds.masks.mean(), test_ds.masks.mean()

(tensor(0.0392, device='cuda:0'), tensor(0.0511, device='cuda:0'))

## RNN

In [26]:
class ImageFeatureExtractor(nn.Module):
    def __init__(
        self,
        img_tensor,
        input_size=32,
        output_dim=32,
        dropout=0.2,
    ):
        super(ImageFeatureExtractor, self).__init__()
        
        self.input_size = 32
        self.output_dim = output_dim
        self.img_tensor = img_tensor.to(device) # (900, 3, 32, 32)
        
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3), # 30x30
            nn.ReLU(),
            nn.MaxPool2d(2), # 15x15
            nn.Conv2d(16, 32, 4), # 12x12
            nn.BatchNorm2d(32),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.MaxPool2d(2), # 6x6
            nn.Conv2d(32, 32, 3), # 4x4
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.MaxPool2d(2), # 2x2
            nn.Flatten(),
            nn.Linear(4*32, output_dim),
            nn.Dropout(dropout),
            nn.ReLU()
        ).to(device)
    
    def forward(self):
        features = self.model(self.img_tensor) # (900, feature_embedding)
        features = features.view(CFG.grid_size, CFG.grid_size, -1)
        features = torch.permute(features, (2, 0, 1)).unsqueeze(0)
        return features

class ConvGRULayer(nn.Module):
    def __init__(self, input_size, hidden_size, kernel_size, dilation = 1, dropout=0.2):
        super(ConvGRULayer, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.kernel_size = kernel_size
        padding = ((kernel_size - 1) * dilation) // 2

        self.conv_ur = nn.Conv2d(
            input_size + hidden_size, hidden_size + hidden_size, kernel_size, padding=padding,
            dilation=dilation
        ).to(device)

        self.conv_h = nn.Conv2d(input_size + hidden_size, hidden_size, kernel_size, 
                                padding=padding, dilation=dilation).to(device)

        # Dropout doesn't work when high spatial correlation between adjacent cells - use Dropout2d
        self.dropout = nn.Dropout2d(dropout).to(device)
        self.init_weights_()

    def init_weights_(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    
    def forward(self, x, h=None):

        B, C, H, W = x.shape

        if h is None:
            h = torch.zeros(B, self.hidden_size, H, W).to(device)

        xh = torch.concatenate([x, h], dim = 1)
        update_reset = self.conv_ur(xh)
        update_reset = F.sigmoid(update_reset)
        update = update_reset[:, :self.hidden_size]
        reset = update_reset[:, self.hidden_size:]

        x_rh = torch.concatenate([x, reset * h], dim = 1)
        candidate_h = self.conv_h(x_rh)
        candidate_h = F.tanh(candidate_h)

        h_new = (1 - update) * h + update * candidate_h
        h_new = self.dropout(h_new)

        return h_new

class ConvGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, kernel_size=3, atrous=False, dropout=0.2):
        super(ConvGRU, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.layers = []
        for i in range(num_layers):
            if i == 0:
                self.layers.append(
                    ConvGRULayer(input_size, hidden_size, kernel_size, dropout=dropout).to(device)
                )
            else:
                if atrous and i % 2 == 1:
                    self.layers.append(
                        ConvGRULayer(hidden_size, hidden_size, kernel_size, 
                                     dilation= 2, dropout=dropout).to(device)
                    )
                else:
                    self.layers.append(
                        ConvGRULayer(hidden_size, hidden_size, kernel_size, dropout=dropout).to(device)
                    )

    def init_hidden(self, x):
        B, T, C, H, W = x.shape # batch, time, channels, h, w
        # h_states = torch.zeros(self.num_layers, B, self.hidden_size, H, W)
        h_states = [None] * self.num_layers
        return h_states
    
    def forward(self, x):
        B, T, C, H, W = x.shape # batch, time, channels, h, w
        h_states = self.init_hidden(x)
        outputs = []
        
        for t in range(T):
            xt = x[:, t]
            for l, layer in enumerate(self.layers):
                h_states[l] = layer(xt, h_states[l])
                xt = h_states[l]

            outputs.append(h_states[-1].unsqueeze(1))

        return torch.cat(outputs, dim=1)

class ConvPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, img_tensor, dropout=0.2,
                 kernel_size=3, output_size=2, forecast_horizon=35):
        """
        ConvGRU based model to inpaint AQI predictions
        """
        super(ConvPredictor, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.forecast_horizon = forecast_horizon

        self.conv_gru = ConvGRU(
            input_size, hidden_size, num_layers, kernel_size=kernel_size, atrous=False, dropout=dropout
        ).to(device)

        self.feature_extractor = ImageFeatureExtractor(img_tensor).to(device)
        
        self.conv_predictor = nn.Conv2d(hidden_size + 32, forecast_horizon * output_size, 1).to(device)

        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize weights using Xavier for RNN and linear layers."""
        for name, param in self.named_parameters():
            if 'weight_ih' in name:  # Input-hidden weights
                nn.init.xavier_uniform_(param)
            elif 'weight_hh' in name:  # Hidden-hidden weights
                nn.init.orthogonal_(param)  # Orthogonal init for stability
            elif 'bias' in name:
                nn.init.zeros_(param)  # Zero bias for stability
            elif 'fc' in name:  # Fully connected layer
                nn.init.kaiming_uniform_(param, nonlinearity='relu')

    def forward(self, x):
        B = x.size(0)

        features = self.feature_extractor() # (1, 32, 30, 30)
        features = features.repeat(B, 1, 1, 1)# (B, 32, 30, 30)
        gru_out = self.conv_gru(x)
        gru_final_hidden = gru_out[:, -1]
        final_features = torch.cat([gru_final_hidden, features], dim = 1)
        out = self.conv_predictor(final_features)
        
        out = out.view(B, self.forecast_horizon, self.output_size, CFG.grid_size, CFG.grid_size)
        
        return out

def get_optimizer(model, lr = 1e-4, weight_decay = 1e-5):
    optimizer = Adam(params = model.parameters(), 
                     lr = lr, weight_decay = weight_decay)
    return optimizer

def get_reduce_lr(optimizer, factor=0.1, patience=2):
    scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = factor, patience = patience)
    return scheduler

In [27]:
def get_model(
    img_tensor, 
    input_size=None, 
    hidden_size=32, 
    num_layers=3, 
    dropout=0.2,
    kernel_size=3, 
    output_size=2, 
    forecast_horizon=35
):
    if input_size is None:
        input_size = len(CFG.feature_indices)
        
    model = ConvPredictor(
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout,
        output_size=output_size, 
        forecast_horizon=forecast_horizon,
        img_tensor = img_tensor
    )
    return model


## Train and Validation Functions

In [28]:
class MaskedMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super(MaskedMSELoss, self).__init__()
        self.eps = eps

    def forward(self, preds, targets, mask):
        loss = (preds - targets) ** 2
        masked_loss = (loss * mask).sum() / (mask.sum() + self.eps)
        return masked_loss

class MaskedMAELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super(MaskedMAELoss, self).__init__()
        self.eps = eps

    def forward(self, preds, targets, mask):
        loss = abs(preds - targets)
        masked_loss = (loss * mask).sum() / (mask.sum() + self.eps)
        return masked_loss

class MaskedR2(nn.Module):
    def __init__(self, eps=1e-6):
        super(MaskedR2, self).__init__()
        self.eps = eps

    def forward(self, preds, targets, mask):
        error = ((targets - preds) ** 2) * mask
        masked_mse = error.sum() / (mask.sum() + self.eps)
        mean_y = (targets * mask).sum() / (mask.sum() + self.eps)
        total_var = (((targets - mean_y) ** 2) * mask).sum() / (mask.sum() + self.eps)
        r2 = 1 - (masked_mse / (total_var + self.eps))
        return r2

masked_metrics_dict = {
    'MSE': MaskedMSELoss(),
    'MAE': MaskedMAELoss(),
    'r2 score': MaskedR2(),
}

In [29]:
class RunningLoss:
    def __init__(self, window = 10):
        self.loss = 0
        self.total = 0
        self.loss_last = []
        self.total_last = []

        self.window = window
    
    def update(self, loss, batch_size):
        total = 1
        self.loss += loss
        self.total += 1
        
        self.loss_last.append(loss)
        self.total_last.append(total)
        if len(self.loss_last) > self.window:
            self.loss_last.pop(0)
            self.total_last.pop(0)
    
    def reset(self):
        self.loss = 0
        self.total = 0
        self.loss_last = []
        self.total_last = []

    def print(self):
        print(f"Accuracy: {self.loss / self.total}")
    
    def get_curr_stats(self):
        return sum(self.loss_last) / sum(self.total_last)

    def total_stats(self):
        return self.loss / self.total


In [30]:
def validate(
    model,
    test_dl,
    criterion,
    verbose=False,
    get_predict=False,
):
    running_loss = RunningLoss()

    predict_array = []
    true_array = []
    mask_array = []
    
    if verbose >= 1:
        pbar = tqdm(total = len(test_dl), ncols = 110, desc = "Validation Progress") # , dynamic_ncols=True, leave=False)
    else:
        pbar = None

    with torch.no_grad():
        for i, (x, y, mask) in enumerate(test_dl):
            x, y, mask = x.to(device), y.to(device), mask.to(device)
            pred = model(x)

            pred = target_scaler.inverse_transform(pred)
            y = target_scaler.inverse_transform(y)

            temp_loss = criterion(pred, y, mask)
            running_loss.update(temp_loss.item(), batch_size=x.shape[0])

            if get_predict:
                predict_array.append(pred)
                true_array.append(y)
                mask_array.append(mask)

            if verbose >= 1:
                total = len(test_dl)
                curr_loss = np.sqrt(running_loss.get_curr_stats())
                pbar.set_description(f"Validation Step {i} / {total}")
                pbar.set_postfix(Loss=f"{curr_loss:.4f}")
                pbar.update(1)
        if verbose >= 1:
            pbar.close()

    if get_predict:
        return predict_array, true_array, mask_array, np.sqrt(running_loss.total_stats())
    
    return np.sqrt(running_loss.total_stats())


In [31]:
def train(
        model,
        criterion,
        optimizer,
        num_epochs,
        train_dataloader,
        val_dataloader,
        scheduler=None,
        do_validate=False,
        validate_frequency=1,
        verbose=True,
        metrics_dict=None,
        model_name="conv_"
):
    if metrics_dict is None:
        metrics_dict = masked_metrics_dict
    
    loss_list = []
    val_loss_list = []
    all_loss_list = []

    best_val_loss = 100000000
    best_val_loss25 = 100000000
    best_val_loss10 = 100000000

    for epoch in tqdm(range(num_epochs), disable=verbose >= -1):
        running_loss = RunningLoss()
        if verbose >= 1:
            pbar = tqdm(total = len(train_dataloader), ncols = 110, desc = "Training Progress") # , dynamic_ncols=True, leave=False)
        else:
            pbar = None
        for i, (x, y, mask) in enumerate(train_dataloader):
            x, y, mask = x.to(device), y.to(device), mask.to(device)

            optimizer.zero_grad()
            
            pred = model(x)
            loss = criterion(pred, y, mask)
            loss.backward()

            optimizer.step()

            all_loss_list.append(loss.item())

            with torch.no_grad():
                pred = target_scaler.inverse_transform(pred)
                y = target_scaler.inverse_transform(y)
                temp_loss = torch.sqrt(criterion(pred, y, mask))
                running_loss.update(temp_loss.item(), batch_size=x.shape[0])

            if verbose >= 1:
                total = len(train_dataloader)
                curr_loss = running_loss.get_curr_stats()
                pbar.set_description(f"Epoch {epoch}: Step {i} / {total}")
                pbar.set_postfix(Loss=f"{curr_loss:.4f}")
                pbar.update(1)
        if verbose >= 1:
            print(f"Train Loss Total: {running_loss.total_stats()}")
            pbar.close()
        
        if do_validate and epoch % validate_frequency == validate_frequency - 1:
            pred1, y1, mask1, val_loss = validate(model, val_dataloader, criterion, 
                                           verbose=verbose, get_predict=True)
            pred1 = torch.concat(pred1)
            y1 = torch.concat(y1)
            mask1 = torch.concat(mask1)
            
            val_loss_list.append(val_loss)
            if scheduler is not None:
                scheduler.step(val_loss)
            
            if verbose >= 1:
                print(f"Val Loss Total: {val_loss}")

            if val_loss < best_val_loss:
                if verbose >= 0:
                    print(f"Better Val Loss: {val_loss} < {best_val_loss}")
                best_val_loss = val_loss
                torch.save(model.state_dict(), f"./{model_name}best.pth")

            with torch.no_grad():
                for i, t in enumerate(CFG.target_list):
                    for n, m in metrics_dict.items():
                        l = m(pred1[:, :, i, :, :], y1[:, :, i, :, :], mask1[:, :, 0, :, :]).cpu().detach()
                        l = float(l)
                        if n == 'MSE':
                            # print(y1[:, :, i].shape, pred1[:, :, i].shape, mask1.shape)
                            mse_loss = np.sqrt(l)
                            if verbose >= 1:
                                print(f"{t}: {n}: {mse_loss}")
                            
                            if i == 0 and mse_loss < best_val_loss25:
                                if verbose >= 0:
                                    print(f"----------Better MSE on pm2_5 {mse_loss} < {best_val_loss25}")
                                torch.save(model.state_dict(), f"./{model_name}2_5.pth")
                                best_val_loss25 = mse_loss
                            if i == 1 and mse_loss < best_val_loss10:
                                if verbose >= 0:
                                    print(f"----------Better MSE on pm10 {mse_loss} < {best_val_loss10}")
                                torch.save(model.state_dict(), f"./{model_name}10.pth")
                                best_val_loss10 = mse_loss
                        if verbose >= 1:
                            if n == 'MSE':
                                l = np.sqrt(l)
                                l = float(l)
                            print(f"{t}: {n}: {l}")
        
        epoch_loss = running_loss.total_stats()
        loss_list.append(epoch_loss)
    
    return loss_list, val_loss_list, all_loss_list


## Run

In [32]:
class SafeSchedulerWrapper:
    def __init__(self, scheduler):
        self.scheduler = scheduler

    def step(self, metric=None):
        self.scheduler.step()

def cosine_scheduler_with_warmup(optimizer, warmup_steps, total_steps, base_lr, final_lr=0.0):
    def lr_lambda(current_step):
        if current_step < warmup_steps:
            return current_step / warmup_steps
        progress = (current_step - warmup_steps) / (total_steps - warmup_steps)
        cosine_decay = 0.5 * (1 + math.cos(math.pi * progress))
        return final_lr / base_lr + (1 - final_lr / base_lr) * cosine_decay

    return SafeSchedulerWrapper(torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda))

In [33]:
model = get_model(
    img_tensor=train_ds.imgs,
    hidden_size = CFG.hidden_dim,
    num_layers=CFG.num_layers, 
    dropout=CFG.dropout,
    kernel_size=CFG.kernel_size, 
    output_size=2, 
    forecast_horizon=CFG.forecast_horizon
).to(device)

criterion = MaskedMSELoss().to(device)
optimizer = get_optimizer(model, lr = CFG.lr)
scheduler = get_reduce_lr(optimizer, factor=CFG.factor, patience=CFG.patience)
# cosine_scheduler_with_warmup = cosine_scheduler_with_warmup(
#     optimizer, CFG.warmup_steps, CFG.total_steps, CFG.base_lr, CFG.final_lr
# )

In [34]:
# validate(model, test_dl, criterion, verbose=True)

In [35]:
# # loss_list, val_loss_list, all_loss_list
# # pred1, y1, 
# loss_list, val_loss_list, all_loss_list = train(
#     model,
#     criterion,
#     optimizer,
#     num_epochs=CFG.num_epochs,
#     train_dataloader=train_dl,
#     val_dataloader=test_dl,
#     scheduler=scheduler,
#     do_validate=True,
#     verbose=-2,
# )

In [36]:
def get_aqi_category_indices(preds, thresholds):
    indices = torch.bucketize(preds, torch.tensor(thresholds).to(device), right=False) - 1
    return torch.clamp(indices, min=0, max=5)

def compute_aqi_classification_metrics(pred, label, mask):
    mask = mask.bool().squeeze(2)

    metrics = {}
    for i, (name, thresholds) in enumerate(zip(CFG.target_list, [CFG.pm2_5_thresholds, CFG.pm10_thresholds])):
        pred_class = get_aqi_category_indices(pred[:, :, i, :, :], thresholds)
        label_class = get_aqi_category_indices(label[:, :, i, :, :], thresholds)

        pred_class = pred_class[mask]
        label_class = label_class[mask]

        class_metrics = {}
        for class_idx, class_name in enumerate(CFG.aqi_category):
            true_pos = ((pred_class == class_idx) & (label_class == class_idx)).sum().item()
            total_pred = (pred_class == class_idx).sum().item()
            total_true = (label_class == class_idx).sum().item()

            accuracy = true_pos / total_true if total_true else 0.0
            precision = true_pos / total_pred if total_pred else 0.0
            recall = true_pos / total_true if total_true else 0.0

            class_metrics[class_name] = {
                "true": true_pos,
                "total": total_true,
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall
            }

        metrics[name] = class_metrics

    return metrics


In [37]:
def predict_eval(model, test_dl, criterion, verbose=True):
    o = validate(model, test_dl, criterion, verbose=verbose, get_predict=True)
    pred = torch.concat(o[0])
    y = torch.concat(o[1])
    mask = torch.concat(o[2])
    model_performance = {'pm2_5': {}, 'pm10': {}}
    for i, t in enumerate(CFG.target_list):
        for n, m in masked_metrics_dict.items():
            l = m(pred[:, :, i, :, :], y[:, :, i, :, :], mask[:, :, 0, :, :])
            if n == 'MSE':
                l = np.sqrt(l.cpu().numpy())
                if verbose:
                    print(f"{t}: {n}: {l}")
            else:
                if verbose:
                    print(f"{t}: {n}: {l}")

            model_performance[t][n] = float(l)
    
    model_performance['classification'] = compute_aqi_classification_metrics(pred, y, mask)
    
    return model_performance

In [38]:
# model_sd = torch.load('/kaggle/working/conv_best.pth')
# model.load_state_dict(model_sd)

In [39]:
# predict_eval(model, train_dl, criterion, verbose=True)
# print()

## Equivalent Evaluation

In [40]:
full_ds = np.load('/kaggle/input/airdelhi-baselines-deepengineering/idw_full_dense_grid.npz', allow_pickle=True)

full_df = full_ds['grids']
full_k = full_ds['keys']

rolled_df = np.roll(full_df, shift=35*7, axis=0)
full_df = np.concatenate([full_df, rolled_df[:, :2]], axis = 1)
full_df[:7*35, 7:, :, :] = np.mean(full_df[:7*35, :2], axis = 0)

full_np_df = full_df
full_np_df[:, 0, :, :] = target_scaler.transform_numpy(full_np_df[:, 0, :, :], 'pm2_5')
full_np_df[:, 1, :, :] = target_scaler.transform_numpy(full_np_df[:, 1, :, :], 'pm10')
full_np_df[:, 7, :, :] = target_scaler.transform_numpy(full_np_df[:, 7, :, :], 'pm2_5')
full_np_df[:, 8, :, :] = target_scaler.transform_numpy(full_np_df[:, 8, :, :], 'pm10')

feature_indices = [2, 3, 4, 5]
def min_max_scale(d):
    d = (d - d.min()) / (d.max() - d.min())
    return d

for f in feature_indices:
    full_np_df[:, f, :, :] = min_max_scale(full_np_df[:, f, :, :])    

full_train_ds = ConvImageSequenceDataset(full_np_df[:max_train_index, :, :, :], img_dict)
full_test_ds = ConvImageSequenceDataset(full_np_df[max_train_index:, :, :, :], img_dict)

full_train_dl = DataLoader(full_train_ds, batch_size = CFG.batch_size, shuffle=True)
full_test_dl = DataLoader(full_test_ds, batch_size = CFG.batch_size, shuffle=True)


# Ablation Study

## Ablation 1: Model Size

In [41]:
import itertools
import json

def run_grid_search(
    param_grid: dict,
        df_dict,
        model_name,
        img_tensor,
    model_save_dir: str,
    result_json_path: str,
    default_params: dict = None
):
    from copy import deepcopy

    # Generate all parameter combinations
    keys, values = zip(*param_grid.items())
    
    param_combinations = [
        {**(default_params or {}), **dict(zip(keys, v))}
        for v in itertools.product(*values)
    ]
    
    model_dict = {}
    results_list = []
    
    for idx, param_set in enumerate(param_combinations):
        print(f"Training model {idx} with params: {param_set}")
        
        model = get_model(img_tensor=img_tensor, **param_set)
        criterion = MaskedMSELoss().to(device)
        optimizer = get_optimizer(model, lr = CFG.lr)
        scheduler = get_reduce_lr(optimizer, factor=CFG.factor, patience=CFG.patience)
        
        loss_list, val_loss_list, all_loss_list = train(
            model,
            criterion,
            optimizer,
            num_epochs=CFG.num_epochs,
            train_dataloader=df_dict['train_dl'],
            val_dataloader=df_dict['test_dl'],
            scheduler=scheduler,
            do_validate=True,
            validate_frequency=1,
            verbose=-2,
            model_name=model_save_dir + f"{model_name}_{idx}_"
        )
        
        model_path = model_save_dir + f"final_{model_name}_{idx}.pt"
        torch.save(model.state_dict(), model_path)
        model_dict[idx] = model

        result_metrics = {}
        paths = ['best', '2_5', '10']
        for p in paths:
            result_metrics[p] = {}
            model_state_dict = torch.load(model_save_dir + f"{model_name}_{idx}_{p}.pth")
            model.load_state_dict(model_state_dict)
            
            for df_name, df1 in df_dict.items():
                result_metrics[p][df_name] = predict_eval(model, df1, criterion, verbose=False)

        results_list.append({
            "model_index": idx,
            "model_name": model_path,
            "parameters": deepcopy(param_set),
            "results": result_metrics
        })

    with open(result_json_path, "w") as f:
        json.dump(results_list, f, indent=4)

    return model_dict


In [42]:
param_grid = {
    # 'hidden_size' : [16, 32, 64, 128], 
    'hidden_size' : [64], 
    'num_layers' : [1, 3, 6, 9],
}

default_params = {
    'input_size' : None, 
    'forecast_horizon' : 35
}

df_dict = {
    'train_dl' : train_dl,
    'test_dl' : test_dl,
    'full_train_dl' : full_train_dl,
    'full_test_dl' : full_test_dl,
}

model_name = "model_size_test"

model_save_dir = f"./{model_name}_dir/"
result_json_path = f"{model_save_dir}{model_name}_results.json"
os.makedirs(model_save_dir, exist_ok=True)

img_tensor = train_ds.imgs

model_dict = run_grid_search(
    param_grid,
        df_dict,
        model_name,
        img_tensor,
    model_save_dir,
    result_json_path,
    default_params
)

Training model 0 with params: {'input_size': None, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 1}


100%|██████████| 50/50 [01:32<00:00,  1.85s/it]
<ipython-input-41-240d09f39090>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model 1 with params: {'input_size': None, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 3}


100%|██████████| 50/50 [03:59<00:00,  4.79s/it]


Training model 2 with params: {'input_size': None, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 6}


100%|██████████| 50/50 [07:53<00:00,  9.46s/it]


Training model 3 with params: {'input_size': None, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 9}


100%|██████████| 50/50 [11:47<00:00, 14.16s/it]


## Ablation 2: Sequence Length 

In [43]:
def get_dl(df, sequence_length=35, forecast_horizon=35):
    ds = ConvImageSequenceDataset(df, 
                              sequence_length = sequence_length, forecast_horizon = forecast_horizon,
                             img_dict=img_dict)
    dl = DataLoader(ds, batch_size = CFG.batch_size, shuffle=True)
    return dl

In [44]:
def run_grid_search_df(
    param_grid: dict,
        df_dict,
        model_name,
        img_tensor,
    model_save_dir: str,
    result_json_path: str,
    default_params: dict = None,
    model_params: dict = None,
):
    from copy import deepcopy

    # Generate all parameter combinations
    keys, values = zip(*param_grid.items())
    
    param_combinations = [
        {**(default_params or {}), **dict(zip(keys, v))}
        for v in itertools.product(*values)
    ]
    
    model_dict = {}
    results_list = []
    
    for idx, param_set in enumerate(param_combinations):
        print(f"Training model on dataset {idx} with params: {param_set}")

        model = get_model(img_tensor = img_tensor, **model_params)

        ds_dict = {
            k : get_dl(df = v, **param_set)
            for k, v in df_dict.items()
        }
        
        criterion = MaskedMSELoss().to(device)
        optimizer = get_optimizer(model, lr = CFG.lr)
        scheduler = get_reduce_lr(optimizer, factor=CFG.factor, patience=CFG.patience)
        
        loss_list, val_loss_list, all_loss_list = train(
            model,
            criterion,
            optimizer,
            num_epochs=CFG.num_epochs,
            train_dataloader=ds_dict['train_dl'],
            val_dataloader=ds_dict['test_dl'],
            scheduler=scheduler,
            do_validate=True,
            validate_frequency=1,
            verbose=-2,
            model_name=model_save_dir + f"{model_name}_{idx}_"
        )
        
        model_path = model_save_dir + f"final_{model_name}_{idx}.pth"
        torch.save(model.state_dict(), model_path)
        model_dict[idx] = model
        
        result_metrics = {}
        paths = ['best', '2_5', '10']
        for p in paths:
            result_metrics[p] = {}
            model_state_dict = torch.load(model_save_dir + f"{model_name}_{idx}_{p}.pth")
            model.load_state_dict(model_state_dict)
            
            for df_name, df1 in ds_dict.items():
                result_metrics[p][df_name] = predict_eval(model, df1, criterion, verbose=False)

        results_list.append({
            "model_index": idx,
            "model_name": model_path,
            "parameters": deepcopy(param_set),
            "results": result_metrics
        })

    with open(result_json_path, "w") as f:
        json.dump(results_list, f, indent=4)

    return model_dict


In [45]:
param_grid = {
    'sequence_length' : [35, 70, 105, 140, 175], 
}

default_params = {
    'input_size' : None, 
    'forecast_horizon' : 35
}

model_params = {
    'input_size': None,
    'forecast_horizon': 35,
    'num_layers': 3,
    'hidden_size': 64,
}

df_dict = {
    'train_dl' : np_df[:max_train_index, :, :, :],
    'test_dl' : np_df[max_train_index:, :, :, :],
    'full_train_dl' : full_np_df[:max_train_index, :, :, :],
    'full_test_dl' : full_np_df[max_train_index:, :, :, :],
}

model_name = "sequence_length_test"

model_save_dir = f"./{model_name}_dir/"
result_json_path = f"{model_save_dir}{model_name}_results.json"
os.makedirs(model_save_dir, exist_ok=True)

model_dict1 = run_grid_search_df(
    param_grid,
        df_dict,
        model_name,
        img_tensor,
    model_save_dir,
    result_json_path,
    default_params=None,
    model_params = model_params,
)

Training model on dataset 0 with params: {'sequence_length': 35}


100%|██████████| 50/50 [02:12<00:00,  2.65s/it]
<ipython-input-44-98c7dc03ae99>:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 1 with params: {'sequence_length': 70}


100%|██████████| 50/50 [04:08<00:00,  4.96s/it]


Training model on dataset 2 with params: {'sequence_length': 105}


100%|██████████| 50/50 [05:44<00:00,  6.88s/it]


Training model on dataset 3 with params: {'sequence_length': 140}


100%|██████████| 50/50 [07:28<00:00,  8.97s/it]


Training model on dataset 4 with params: {'sequence_length': 175}


100%|██████████| 50/50 [09:14<00:00, 11.09s/it]


## Ablation 3: Forecast Horizon

In [46]:
def run_grid_search_horizon(
    horizon_values,
        df_dict,
        model_name,
        img_tensor,
    model_save_dir: str,
    result_json_path: str,
    df_params: dict = None,
    model_params: dict = None,
):    
    model_dict = {}
    results_list = []
    
    for idx, h in enumerate(horizon_values):
        print(f"Training model on dataset {idx} with horizon: {h}")

        model = get_model(img_tensor=img_tensor, forecast_horizon=h, **model_params)

        ds_dict = {
            k : get_dl(df = v, forecast_horizon = h, **df_params)
            for k, v in df_dict.items()
        }
        
        criterion = MaskedMSELoss().to(device)
        optimizer = get_optimizer(model, lr = CFG.lr)
        scheduler = get_reduce_lr(optimizer, factor=CFG.factor, patience=CFG.patience)
        
        loss_list, val_loss_list, all_loss_list = train(
            model,
            criterion,
            optimizer,
            num_epochs=CFG.num_epochs,
            train_dataloader=ds_dict['train_dl'],
            val_dataloader=ds_dict['test_dl'],
            scheduler=scheduler,
            do_validate=True,
            validate_frequency=1,
            verbose=-2,
            model_name=model_save_dir + f"{model_name}_{idx}_"
        )
        
        model_path = model_save_dir + f"final_{model_name}_{idx}.pt"
        torch.save(model.state_dict(), model_path)
        model_dict[idx] = model
        
        result_metrics = {}
        paths = ['best', '2_5', '10']
        for p in paths:
            result_metrics[p] = {}
            model_state_dict = torch.load(model_save_dir + f"{model_name}_{idx}_{p}.pth")
            model.load_state_dict(model_state_dict)
            
            for df_name, df1 in ds_dict.items():
                result_metrics[p][df_name] = predict_eval(model, df1, criterion, verbose=False)

        results_list.append({
            "model_index": idx,
            "model_name": model_path,
            "horizon": h,
            "params" : model_params,
            "results": result_metrics
        })

    with open(result_json_path, "w") as f:
        json.dump(results_list, f, indent=4)

    return model_dict


In [47]:
horizon_values = [35, 70, 105, 140, 175]

model_params = {
    'input_size': None,
    'num_layers': 3,
    'hidden_size': 64,
}

df_dict = {
    'train_dl' : np_df[:max_train_index, :, :, :],
    'test_dl' : np_df[max_train_index:, :, :, :],
    'full_train_dl' : full_np_df[:max_train_index, :, :, :],
    'full_test_dl' : full_np_df[max_train_index:, :, :, :],
}

df_params = {
    'sequence_length' : 105,
}

model_name = "forecast_horizon_test"

model_save_dir = f"./{model_name}_dir/"
result_json_path = f"{model_save_dir}{model_name}_results.json"
os.makedirs(model_save_dir, exist_ok=True)

model_dict3 = run_grid_search_horizon(
    horizon_values,
        df_dict,
        model_name,
        img_tensor,
    model_save_dir,
    result_json_path,
    df_params=df_params,
    model_params = model_params,
)

Training model on dataset 0 with horizon: 35


100%|██████████| 50/50 [05:20<00:00,  6.41s/it]
<ipython-input-46-706c988bf109>:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 1 with horizon: 70


100%|██████████| 50/50 [05:12<00:00,  6.24s/it]


Training model on dataset 2 with horizon: 105


100%|██████████| 50/50 [05:13<00:00,  6.27s/it]


Training model on dataset 3 with horizon: 140


100%|██████████| 50/50 [05:11<00:00,  6.23s/it]


Training model on dataset 4 with horizon: 175


100%|██████████| 50/50 [04:55<00:00,  5.90s/it]


In [48]:
# with open("/kaggle/working/model_size_test_dir/model_size_test_results.json", "r") as f:
#     j = json.load(f)